In [1]:
# https://pytorch.org/tutorials/beginner/transfer_learning_tutorial.html

# This Python 3 environment comes with many helpful analytics libraries installed
# It is defined by the kaggle/python Docker image: https://github.com/kaggle/docker-python
# For example, here's several helpful packages to load

import numpy as np # linear algebra
import pandas as pd # data processing, CSV file I/O (e.g. pd.read_csv)

# Input data files are available in the read-only "../input/" directory
# For example, running this (by clicking run or pressing Shift+Enter) will list all files under the input directory

import os

# You can write up to 5GB to the current directory (/kaggle/working/) that gets preserved as output when you create a version using "Save & Run All" 
# You can also write temporary files to /kaggle/temp/, but they won't be saved outside of the current session

import time
import copy
import matplotlib.pyplot as plt
from PIL import Image
from barbar import Bar

import torch
import torch.nn as nn
import torch.optim as optim
from torch.optim import lr_scheduler
from torch.utils.data import Dataset
import torchvision
from torchvision import datasets, transforms, models
from efficientnet_pytorch import EfficientNet

In [2]:
input_root = './shopee-product-detection-dataset'

train_df = pd.read_csv(os.path.join(input_root, 'train.csv'))
test_df = pd.read_csv(os.path.join(input_root, 'test.csv'))

In [3]:
class ShopeeTestDataset(Dataset):
    def __init__(self, train_file_lt, transform=None):
        self.imgs = train_file_lt
        self.transform = transform
    
    def __getitem__(self, idx):
        img_filename = self.imgs[idx]
        img = Image.open(os.path.join(input_root, 'test', 'test', img_filename))
        if self.transform is not None:
            img = self.transform(img)
            
        filename = self.imgs[idx]
        return img, filename
    
    def __len__(self):
        return len(self.imgs)

# Data augmentation and normalization for training
# Just normalization for validation
data_transforms = {
    'train': transforms.Compose([
        # https://discuss.pytorch.org/t/randomresizedcrop-vs-resize-updated/35357/2
        transforms.RandomResizedCrop(224),
        transforms.RandomHorizontalFlip(),
        transforms.ToTensor(),
        transforms.Normalize([0.485, 0.456, 0.406], [0.229, 0.224, 0.225])
    ]),
    'test': transforms.Compose([
        transforms.Resize(256),
        transforms.CenterCrop(224),
        transforms.ToTensor(),
        transforms.Normalize([0.485, 0.456, 0.406], [0.229, 0.224, 0.225])
    ]),
}

test_filenames = test_df['filename'].tolist()
image_datasets = {'train': datasets.ImageFolder(os.path.join(input_root, 'train/train'),
                                          data_transforms['train']),
                  'test': ShopeeTestDataset(test_filenames, transform=data_transforms['test'])}
dataloaders = {'train': torch.utils.data.DataLoader(image_datasets['train'], batch_size=256,
                                              shuffle=True, num_workers=4),
               'test': torch.utils.data.DataLoader(image_datasets['test'], batch_size=512,
                                              shuffle=False, num_workers=0)}
dataset_sizes = {x: len(image_datasets[x]) for x in ['train', 'test']}
class_names = image_datasets['train'].classes

device = torch.device("cuda:0" if torch.cuda.is_available() else "cpu")
print(device)

cuda:0


In [4]:
def imshow(inp, title=None):
    """Imshow for Tensor."""
    inp = inp.numpy().transpose((1, 2, 0))
    mean = np.array([0.485, 0.456, 0.406])
    std = np.array([0.229, 0.224, 0.225])
    inp = std * inp + mean
    inp = np.clip(inp, 0, 1)
    plt.imshow(inp)
    if title is not None:
        plt.title(title)
    plt.pause(0.001)  # pause a bit so that plots are updated


# Get a batch of training data
# inputs, classes = next(iter(dataloaders['train']))

# Make a grid from batch
# out = torchvision.utils.make_grid(inputs)

# imshow(out, title=[class_names[x] for x in classes])

In [5]:
model_ft = EfficientNet.from_pretrained('efficientnet-b1', num_classes=42) # https://github.com/lukemelas/EfficientNet-PyTorch
# True Unfreeze model weights https://www.kaggle.com/ateplyuk/pytorch-efficientnet
# for param in model_ft.parameters():
#     param.requires_grad = True

# num_ftrs = model_ft._fc.in_features
# Here the size of each output sample is set to 42.
# Alternatively, it can be generalized to nn.Linear(num_ftrs, len(class_names)).
# model_ft._fc = nn.Linear(num_ftrs, 42)

model_ft = model_ft.to(device)

Loaded pretrained weights for efficientnet-b1


In [6]:
def train_model(model, criterion, optimizer, scheduler, num_epochs=25):
    since = time.time()

    best_model_wts = copy.deepcopy(model.state_dict())
    best_acc = 0.0

    for epoch in range(num_epochs):
        print('Epoch {}/{}'.format(epoch, num_epochs - 1))
        print('-' * 10)

        # Each epoch has a training and validation phase
        for phase in ['train']:
            if phase == 'train':
                model.train()  # Set model to training mode
            else:
                model.eval()   # Set model to evaluate mode

            running_loss = 0.0
            running_corrects = 0

            # Iterate over data.
            for idx, (inputs, labels) in enumerate(Bar(dataloaders[phase])):
                inputs = inputs.to(device)
                labels = labels.to(device)

                # zero the parameter gradients
                optimizer.zero_grad()

                # forward
                # track history if only in train
                with torch.set_grad_enabled(phase == 'train'):
                    outputs = model(inputs)
                    _, preds = torch.max(outputs, 1)
                    loss = criterion(outputs, labels)

                    # backward + optimize only if in training phase
                    if phase == 'train':
                        loss.backward()
                        optimizer.step()

                # statistics
                running_loss += loss.item() * inputs.size(0)
                running_corrects += torch.sum(preds == labels.data)
            if phase == 'train':
                scheduler.step()

            epoch_loss = running_loss / dataset_sizes[phase]
            epoch_acc = running_corrects.double() / dataset_sizes[phase]

            print('{} Loss: {:.4f} Acc: {:.4f}'.format(
                phase, epoch_loss, epoch_acc))

            # deep copy the model
            if phase == 'val' and epoch_acc > best_acc:
                best_acc = epoch_acc
                best_model_wts = copy.deepcopy(model.state_dict())

        print()

    time_elapsed = time.time() - since
    print('Training complete in {:.0f}m {:.0f}s'.format(
        time_elapsed // 60, time_elapsed % 60))
    print('Best val Acc: {:4f}'.format(best_acc))

    # load best model weights
    model.load_state_dict(best_model_wts)
    return model

criterion = nn.CrossEntropyLoss()

# Observe that all parameters are being optimized
optimizer_ft = optim.Adam(model_ft.parameters(), lr=1e-1)

# Decay LR by a factor of 0.1 every 5 epochs
exp_lr_scheduler = lr_scheduler.StepLR(optimizer_ft, step_size=5, gamma=0.1)

model_ft = train_model(model_ft, criterion, optimizer_ft, exp_lr_scheduler,
                       num_epochs=25)

Epoch 0/24
----------
105392/105392: [===============================>] - ETA 7.3ssss
train Loss: 4.3647 Acc: 0.0460

Epoch 1/24
----------
105392/105392: [===============================>] - ETA 4.0ssss
train Loss: 3.4349 Acc: 0.0961

Epoch 2/24
----------
105392/105392: [===============================>] - ETA 4.0ssss
train Loss: 3.2213 Acc: 0.1433

Epoch 3/24
----------
105392/105392: [===============================>] - ETA 3.9ssss
train Loss: 3.0716 Acc: 0.1779

Epoch 4/24
----------
105392/105392: [===============================>] - ETA 4.1ssss
train Loss: 2.9225 Acc: 0.2140

Epoch 5/24
----------
105392/105392: [===============================>] - ETA 4.0ssss
train Loss: 2.6141 Acc: 0.2911

Epoch 6/24
----------
105392/105392: [===============================>] - ETA 4.0ssss
train Loss: 2.5316 Acc: 0.3125

Epoch 7/24
----------
105392/105392: [===============================>] - ETA 4.0ssss
train Loss: 2.4708 Acc: 0.3284

Epoch 8/24
----------
105392/105392: [==================

def visualize_model(model, num_images=6):
    was_training = model.training
    model.eval()
    images_so_far = 0
    fig = plt.figure()

    with torch.no_grad():
        for i, (inputs, labels) in enumerate(dataloaders['test']):
            inputs = inputs.to(device)
            labels = labels.to(device)

            outputs = model(inputs)
            _, preds = torch.max(outputs, 1)

            for j in range(inputs.size()[0]):
                images_so_far += 1
                ax = plt.subplot(num_images//2, 2, images_so_far)
                ax.axis('off')
                ax.set_title('predicted: {}'.format(class_names[preds[j]]))
                imshow(inputs.cpu().data[j])

                if images_so_far == num_images:
                    model.train(mode=was_training)
                    return
        model.train(mode=was_training)

visualize_model(model_ft)

In [7]:
# model_ft.load_state_dict(torch.load('efficientb0.pth'), strict=False)

In [8]:
# https://pytorch.org/tutorials/beginner/saving_loading_models.html
torch.save(model_ft.state_dict(), 'efficientb1.pth')

In [9]:
# https://github.com/pytorch/pytorch/issues/2341#issuecomment-442759206
def submission(model):
    was_training = model.training
    model.eval()

    results = []
    with torch.no_grad():
        for i, (inputs, filenames) in enumerate(dataloaders['test']):
            inputs = inputs.to(device)
            outputs = model(inputs)
            _, preds = torch.max(outputs, 1)
            # https://discuss.pytorch.org/t/get-value-out-of-torch-cuda-float-tensor/2539/7
            # print(preds.data.tolist())
            # print(filenames)
            # print(preds)
            results.extend(preds.data.tolist())
            
        model.train(mode=was_training)
    
    df = pd.DataFrame(data={
        'filename': test_filenames,
        'category': results,
    })
    df['category'] = df['category'].apply(lambda x: str(x).zfill(2))
    df.to_csv(os.path.join('./', 'efficientb1.csv'), index=False)

submission(model_ft)